In [1]:
#Charger les données du dataset boston housing
from sklearn import datasets, linear_model
 
# Charger le dataset de maison boston
boston = datasets.load_boston()
#data represente les prédicteurs et target la cible à prédire
print(boston.data.shape)
print(boston.target.shape)


(506, 13)
(506,)


In [2]:
import numpy as np
#afficher les noms des colonnes features
print(boston.feature_names)
#données sur les 5 premiers enregistrements predicteurs
print(boston.data[:5,:10] )
#données sur les 5 premiers enregistrements cibles
print(boston.target[:5,] )
#max, mean et moyenne
print(min(boston.target ),max(boston.target ), np.mean(boston.target))


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.1850e+00
  6.1100e+01 4.9671e+00 2.0000e+00 2.4200e+02]
 [3.2370e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 6.9980e+00
  4.5800e+01 6.0622e+00 3.0000e+00 2.2200e+02]
 [6.9050e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 7.1470e+00
  5.4200e+01 6.0622e+00 3.0000e+00 2.2200e+02]]
[24.  21.6 34.7 33.4 36.2]
5.0 50.0 22.532806324110677


In [6]:
# Décomposer les données prédicteurs en training/testing  
#ensemble apprentissage et test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(boston.data, boston.target, test_size=0.25,
random_state=33)

In [7]:
#Effectuer un scaling approprié autour de la moyenne et variance
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
scalery = StandardScaler().fit(y_train.reshape(-1, 1))
X_train = scalerX.transform(X_train)
y_train = scalery.transform(y_train.reshape(-1, 1))

# Décomposer les cibles-targets en training/testing  
X_test = scalerX.transform(X_test)
y_test = scalery.transform(y_test.reshape(-1, 1))

In [10]:
# Créer l'objet linear regression  
#On utilise dans la manipulation: SGRegressor
from sklearn import linear_model
clf_sgd = linear_model.SGDRegressor(loss='squared_loss', max_iter=5,tol=None
                                    ,penalty=None, random_state=42)
# Faire apprendre le model en utilisant les training set
from sklearn.utils import column_or_1d
clf_sgd.fit(X_train, y_train.ravel())

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty=None,
       power_t=0.25, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [12]:
# Affichage des coefficient
print('Coefficients du modele: \n', clf_sgd.coef_)
print ("Coefficient de determination sur échantillon apprentissage:",clf_sgd.score(X_train, y_train))
# mean square error
print("Residual sum of squares: %.2f" % np.mean((clf_sgd.predict(X_test) - y_test) ** 2))
from sklearn.cross_validation import  KFold, cross_val_score

Coefficients du modele: 
 [-0.08527595  0.06706144 -0.05032898  0.10874804 -0.07755151  0.38961893
 -0.02485839 -0.20990016  0.08491659 -0.05495108 -0.19854006  0.06126093
 -0.37817963]
Coefficient de determination sur échantillon apprentissage: 0.7436177329830429
Residual sum of squares: 1.74


In [13]:
# créer un iterateur de cross validation k-fold avec k=5  
cv = KFold(X_train.shape[0], 5, shuffle=True, random_state=33)
scores = cross_val_score(clf_sgd, X_train, y_train.ravel(), cv=cv)
print ("Moyenne coefficient de determination avec 5-fold crossvalidation:",np.mean(scores))

Moyenne coefficient de determination avec 5-fold crossvalidation: 0.710809853467987


In [14]:
#mesurer la performance
from sklearn import metrics
y_pred = clf_sgd.predict(X_test)


print ("Coefficient de determination:{0:.3f}".format( metrics.r2_score(y_test, y_pred)))

Coefficient de determination:0.650


In [15]:

import matplotlib.pyplot as plt
y_pred = clf_sgd.predict(X_test)
#on recouvre les valeurs par inversion de valeurs
print("Valeurs test")
print(scalery.inverse_transform(y_test[:20].ravel()))
print("Valeurs predictions")
print(scalery.inverse_transform(y_pred[:20]))

Valeurs test
[20.5  5.6 13.4 12.6 21.2 19.7 32.4 14.8 33.  21.4 30.1 36.   8.4 21.6
 16.3 23.  14.9 14.1 31.1 11.9]
Valeurs predictions
[17.86168902 11.20481844 12.59659411 18.26835226 22.79682648 20.94382003
 36.22456352 15.49643251 25.90565528 19.73100676 26.67818312 36.91162851
  5.54809594 25.27292996  9.23896277 24.43412054 17.71033053 18.49341656
 32.44735562 22.14639134]
